In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import re
import requests
from bs4 import BeautifulSoup

import time
from pathlib import Path

# GetDate
from datetime import datetime #,  date

import glob
import os.path

# Desativacao de warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
#Reimportando arquivo - caso execucao tenha sido pausada
#Identificacao de arquivo mais recente
folder_path = r'./ARQ/'
file_type = r'/*csv'
files = glob.glob(folder_path + file_type)
max_file = max(files, key=os.path.getctime)

path_to_file = max_file.replace('\\','/')

#Em caso de definicao manual usar esta linha
#path_to_file = './ARQ/Listagem_13-22.csv'

listagem = pd.read_csv(path_to_file,
                       usecols= ['Projeto','Ano','Tipo','Ementa','Situacao','Autor','Codigo_Tipo'
                                 ,'Id_Projeto_Camara','Data_Projeto','Tramitacao_Encerrada','Aprovado'])

#Substitundo valores
listagem['Aprovado'] = listagem['Aprovado'].fillna('')
listagem['Tramitacao_Encerrada'] = listagem['Tramitacao_Encerrada'].fillna('')
listagem['Data_Projeto'] = listagem['Data_Projeto'].fillna('')
listagem['Codigo_Tipo'] = listagem['Codigo_Tipo'].fillna('')

listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].fillna(0)
listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].astype(int)

ValueError: Usecols do not match columns, columns expected but not found: ['Tramitacao_Encerrada', 'Ano', 'Data_Projeto', 'Tipo', 'Ementa', 'Id_Projeto_Camara', 'Aprovado', 'Codigo_Tipo', 'Projeto', 'Autor', 'Situacao']

In [3]:
# ADICIONAR CONTROLE DE EXECUÇOES, FALHA E EXPORTAÇAO AUTOMATICA
# Preenchimento do campo Id_Projeto_Camara
count = 1
failed_req = 0
failed = False
id_projeto_camara = 0
max_failed_req = 0
notfound_count, req_count = 0, 0 
export = False
max_req = 50

notfound_count_dt, notfound_count_tram, notfound_count_aprv = 0, 0, 0


#fnd_dt, data_projeto, data, fnd_tramitacao, tramitacao, fim_tramitacao = '', '', '', '', '', ''
#ultimo, aprovado = '', ''
#fnd_tramitacao_f, fnd_tramitacao_e = '', ''
#fnd_tramitacao_compl = []

while count > 0:
    for i in listagem.index:
        if listagem['Id_Projeto_Camara'][i] == 0 and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'): 
            with requests.Session() as session:
                
                proposicao_enc = False
                tipo, ementa, situacao, ementa_df_1, ementa_df_2 = '', '', '', '', ''
                id_projeto_camara = 0
                failed = False
                v_resultado = []
                situacao_proposicao = listagem['Situacao'][i]
                
                # Variaveis de requisicao
                num_proj = listagem['Projeto'][i]
                ano_proj = listagem['Ano'][i]
                tipo_proj = listagem['Codigo_Tipo'][i]

                # Variaveis de resposta
                #status = ''
                
                # Requisicao para o portal de propostas da Camara
                query = {"numero": num_proj, "ano": ano_proj, "codtipo": tipo_proj}
                #querys.append(query)

                try:
                    resposta = session.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)
                    req_count+=1
                    
                except:
                    #Assinala que houve falha
                    failed = True
                
                if failed:
                    failed_req += 1
                    if failed_req < max_failed_req:
                        time.sleep(180)   

                    #print(resposta.status_code)
                
                else:
                    filed_req = 0
                    
                    resp_format = BeautifulSoup(resposta.content, "html.parser")
                    fnd_proposicoes = resp_format.find('table', 
                                                        {'border':'0', 
                                                        'cellpadding':'0', 
                                                        'cellspacing':'0', 
                                                        'height':'100%', 
                                                        'width':'100%'}).table.tr.td.find_all('a', 
                                                                                                {'target':'blank'})

                    #v_resultado = []
                    m_proposicoes = [[None]*6]*(int(len(fnd_proposicoes)/6))
                    
                    for f in fnd_proposicoes:
                        v_resultado.append(f)

                
                    
                    l, c, a = 0, 0, 0
    
                    if v_resultado == []:
                        notfound_count += 1
                        id_projeto_camara = -1
                        
                    elif len(v_resultado) > 6:
                        while a < len(v_resultado):
                            m_proposicoes[l][c] = v_resultado[a]
                            a+=1
    
                            # Testes - percorrendo os calores de linhas, colunas e contador
                            #print((a%6))
                            #print(f'a = {a}')
                            #print(f'l = {l}')
                            #print(f'c = {c}')
    
                            if ((a%6)) == 0:
                        
                                tipo = str(re.search('>(.+?)<', (str(m_proposicoes[l][2]))).group(1))
                                #ementa = str(re.search('block;">(.+?)<', (str(m_proposicoes[l][4]))).group(1))
                                ementa = str(re.search('block;">(.+?)/span',str(m_proposicoes[l][4]).replace('...', '').replace('\r', ' ').replace('\n', ' ').replace(' <', '').replace('<', '')).group(1))
                                situacao = str(re.search('>(.+?)<', (str(m_proposicoes[l][5]))).group(1))
                                
                                ementa_df_1 = str(str(listagem['Ementa'][i]).replace('\r', ' ').replace('\n', ' '))
                                ementa_df_2 = listagem['Ementa'][i]
                                
                                #if tipo == listagem['Tipo'][i] and (ementa in ementa_df_1 or ementa in ementa_df_2):
                                if str(tipo).upper().replace(' ', '') == str(listagem['Tipo'][i]).upper().replace(' ', '') and (ementa in ementa_df_1 or ementa in ementa_df_2):
                                    id_projeto_camara = re.search('.PHP(.+?)TAR', str(m_proposicoes[0][0]).upper().replace(' ', '').replace('?NUM=','').replace('"','')).group(1)
                                    situacao_proposicao = situacao
                                    proposicao_enc = True
                                else:
                                    id_projeto_camara = -1    
                                
    
                                l += 1
                                c = 0
                            else:
                                c += 1
                                    
                            if proposicao_enc:
                                break
                    
                    else:
                        tipo = str(re.search('>(.+?)<', (str(v_resultado[2]))).group(1))
                        situacao = str(re.search('>(.+?)<', (str(v_resultado[5]))).group(1))
                                
                        #if tipo == listagem['Tipo'][i]:
                        if str(tipo).upper().replace(' ', '') == str(listagem['Tipo'][i]).upper().replace(' ', ''):
                            id_projeto_camara = (re.search('.PHP(.+?)TAR', str(v_resultado[0]).upper().replace(' ', '').replace('?NUM=','').replace('"','')).group(1))
                            #id_projeto_camara = re.search('.PHP(.+?)TAR', str(v_resultado[0]).upper().replace(' ', '').replace('?NUM=','').replace('"','')).group(1)
                            situacao_proposicao = situacao
                        else:
                            id_projeto_camara = -1 

            
            listagem['Id_Projeto_Camara'][i] = id_projeto_camara 
            listagem['Situacao'][i] = situacao_proposicao
            session.close()
            #time.sleep(0.5)
        
        
        if (listagem['Data_Projeto'][i] == '' or listagem['Tramitacao_Encerrada'][i] == '' or listagem['Aprovado'][i] == '') and listagem['Id_Projeto_Camara'][i] not in (-1, 0) and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'): 
            with requests.Session() as session:

                id_proj = listagem['Id_Projeto_Camara'][i]
                fnd_dt, data_projeto, data, fnd_tramitacao, tramitacao, fim_tramitacao = '', '', '', '', '', ''
                ultimo, aprovado = '', ''
                fnd_tramitacao_f, fnd_tramitacao_e = '', ''
                fnd_tramitacao_compl = []
                
                # Variaveis de resposta
                #status = ''
                
                # Requisicao para o portal de propostas da Camara
                query = {"num": id_proj}
                #querys.append(query)
                
                try:
                    resposta = session.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)
                    req_count+=1
                    
                except:
                    #Assinala que houve falha
                    failed = True
                
                if failed:
                    failed_req += 1
                    if failed_req < max_failed_req:
                        time.sleep(180) 

                    #print(resposta.status_code)
                
                else:
                    filed_req = 0
                    
                    resp_format = BeautifulSoup(resposta.content, "html.parser")
                    
                    try:
                        # Identificacao da data dentro do retorno HTML
                        fnd_dt = resp_format.find('font', {'face':'courier new,arial,verdana'}
                                                          ).find('table', {'width':'98%', 'border':'0'}
                                                                ).find('td',{'bgcolor':'#FFFFFF'}
                                                                          ).find_all('font',{'size':'2'})
                        data = str(fnd_dt[0]).replace(' ', '').upper()
                        data_projeto = re.search('PERÍODO:(.+?)A', data).group(1)

                    except:
                        data_projeto = '31/12/1999'
                        notfound_count_dt += 1

                    try:
                         # Descrição explícita do fim da tramitação dentro do retorno HTML
                        fnd_tramitacao = resp_format.find('font', {'face':'courier new,arial,verdana'}
                                                          ).find('table', {'width':'98%', 'border':'0'}
                                                                ).find_all('p',{'align':'justify'})#.find('font',
                                                             #                           {'size':'2'})
                        tramitacao = str(fnd_tramitacao[1]).replace('	', '').replace(' ', '').upper()

                        if(re.search('-FIMTRAMITAÇÃO:SIM', tramitacao)) or (re.search('TRANSFORMADOEMNORMA', tramitacao)): 

                            fim_tramitacao = 'SIM'
                        else:

                            fim_tramitacao = 'NAO' 
                    
                    except:
                        fim_tramitacao = 'Informacao nao encontrada' 
                        notfound_count_tram += 1 


                    # Identificacao das informacoes da proposicao e do retorno sobre as atividades de tramitacao, buscando pela identificaçao da aprovacao no HTML
                    if 'Aprovad' in listagem['Situacao'][i] or 'Transformad' in listagem['Situacao'][i]:
                        aprovado = 'SIM'
                        fim_tramitacao = 'SIM'
                    #elif (listagem['Codigo_Tipo'][i] not in ('REQ', 'MOC', 'REP', 'PEDIF', 'PEDFF') and fim_tramitacao == 'NAO') or (listagem['Codigo_Tipo'][i] not in ('REQ', 'MOC', 'REP', 'PEDIF', 'PEDFF') and listagem['Situacao'][i] == 'Arquivada') or ('Retirad' in listagem['Situacao'][i]) or ('Rejeitad' in listagem['Situacao'][i]) or ('Vetad' in listagem['Situacao'][i]):# era 'Vetado Totalmente'\
                    elif (listagem['Codigo_Tipo'][i] not in ('REQ', 'MOC', 'REP', 'PEDIF', 'PEDFF') and fim_tramitacao == 'NAO'):
                        aprovado = 'TRAMITACAO'
                    elif (listagem['Codigo_Tipo'][i] not in ('REQ', 'MOC', 'REP', 'PEDIF', 'PEDFF') and listagem['Situacao'][i] == 'Arquivada') or ('Retirad' in listagem['Situacao'][i]) or ('Rejeitad' in listagem['Situacao'][i]) or ('Vetad' in listagem['Situacao'][i]):# era 'Vetado Totalmente'\
                        aprovado = 'NAO'
                        fim_tramitacao = 'SIM'
                    else:
                        
                        try:
                            # Limpeza para pegar o id do projeto dentro da prefeitura
                            fnd_tramitacao_f = resp_format.find('font', {'face':'courier new,arial,verdana'}
                                                          ).find('table', {'width':'98%', 'border':'0'}
                                                                ).find_all('td',{'bgcolor':'#FFFFFF'}
                                                                          )
                            fnd_tramitacao_e = resp_format.find('font', {'face':'courier new,arial,verdana'}
                                                                  ).find('table', {'width':'98%', 'border':'0'}
                                                                        ).find_all('td',{'bgcolor':'#EEEEEE'}
                                                                                  )

                            maxlen = max(len(fnd_tramitacao_f),len(fnd_tramitacao_e))
                            c = 0 

                            while c < maxlen:
                                if c < len(fnd_tramitacao_f):
                                    fnd_tramitacao_compl.append(fnd_tramitacao_f[c])

                                if c < len(fnd_tramitacao_e):
                                    fnd_tramitacao_compl.append(fnd_tramitacao_e[c])

                                c+=1

                            ultimo = str(fnd_tramitacao_compl[len(fnd_tramitacao_compl)-1]).replace('	', '').replace(' ', '').upper()

                            if(re.search('APROVAD', ultimo)) or (re.search('TRANSFORMADOEMNORMA', ultimo)):
                                aprovado = 'SIM'
                                fim_tramitacao = 'SIM'
                            elif (re.search('MANTIDOOVETO', ultimo)) or (re.search('RETIRAD', ultimo)) or (re.search('REJEITAD', ultimo)) or (re.search('RETIRAD', ultimo)): 
                                aprovado = 'NAO'
                                fim_tramitacao = 'SIM'
                            else:
                                if fim_tramitacao == 'SIM':
                                    aprovado = 'NAO'
                                else:
                                    aprovado = 'TRAMITACAO'
                        
                        except:
                            aprovado = 'Informacao nao encontrada'
                            notfound_count_aprv += 1
                
#                except:
                    #Assinala que houve falha
#                    failed = True
#                    time.sleep(180)
                    #print(resposta.status_code)
                    
            
            listagem['Aprovado'][i] = aprovado
            listagem['Tramitacao_Encerrada'][i] = fim_tramitacao
            listagem['Data_Projeto'][i] = data_projeto

            session.close()
            #time.sleep(0.5)
            
                
#       Se houve X falhas consecutivas ou houve mais de Y requisicoes consecutivas 
        if failed_req > max_failed_req or req_count > max_req:
            break
    
    if failed_req > max_failed_req or req_count > max_req:
        break  
          
    count = 0
    
    for i in listagem.index:
        if listagem['Id_Projeto_Camara'][i] == 0 and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
            #((listagem['Id_Projeto_Camara'][i] == 0 or listagem['Id_Projeto_Camara'][i] == '0') and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA') or ((listagem['Data_Projeto'][i] == '' or listagem['Tramitacao_Encerrada'][i] == '' or listagem['Aprovado'][i] == '') and and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):  
            count +=1


    if count == 0:
        break


now = datetime.now()
dthr_finalizado = now.strftime("%d-%m-%Y_%H-%M-%S")

try:
# Exportacao da listagem no estado atual para arquivo csv
    file = './ARQ/ListagemExport_checkpoint_' + dthr_finalizado
    path_to_file = './'+file+'.csv'
    path = Path(path_to_file)

    exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Autor','Codigo_Tipo', 
                    'Id_Projeto_Camara', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

    if path.is_file():
        exp_header = False
    else:    
        exp_header = True

    listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

    export = True

except:
    export = False


if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, {req_count} requisicoes feitas, verifique a conexao. Fim do processo. Nao encontrados: identificadores = {notfound_count}, data = {notfound_count_dt}, tramitacao = {notfound_count_tram}, aprovador = {notfound_count_aprv}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')
elif req_count > max_req:
    print(f'Processo finalizado em {req_count} requisicoes. Nao encontrados: identificadores = {notfound_count}, data = {notfound_count_dt}, tramitacao = {notfound_count_tram}, aprovador = {notfound_count_aprv}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')
else:
    print(f'Fim do processo, {req_count} requisicoes feitas. Nao encontrados: identificadores = {notfound_count}, data = {notfound_count_dt}, tramitacao = {notfound_count_tram}, aprovador = {notfound_count_aprv}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')

    
print(id_projeto_camara)

Fim do processo, 3 requisicoes feitas. Nao encontrados: identificadores = 0, data = 0, tramitacao = 0, aprovador = 0. Finalizado em: 05-02-2023_14-30-03. Status da exportacao: True.
-1


In [ ]:
display(listagem)

In [ ]:
#Exportacao do dataframe no estado atual

now = datetime.now()
dthr_finalizado = now.strftime("%d-%m-%Y_%H-%M-%S")

try:
#if True:
# Exportacao da listagem no estado atual para arquivo csv
    file = 'ListagemExport_checkpoint_' + dthr_finalizado
    #path_to_file = '../ListagemExp/'+file+'.csv'
    path_to_file = './ARQ/'+file+'.csv'
    path = Path(path_to_file)

    exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Autor','Codigo_Tipo', 
                    'Id_Projeto_Camara', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

    if path.is_file():
        exp_header = False
    else:    
        exp_header = True

    listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

    export = True

#else:
except:
    export = False

    
print(export)

In [ ]:
tem=0
naovaiter=0
naotem=0

#oid = ''
for i in listagem.index:
#  if listagem['Id_Projeto_Camara'][i]!=0 and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
#  if listagem['Data_Projeto'][i]!='':
#    if listagem['Codigo_Tipo'][i]!='EMENDA':
    if listagem['Data_Projeto'][i] != '' and listagem['Tramitacao_Encerrada'][i] != '' and listagem['Aprovado'][i] != '' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
        tem+=1
    elif listagem['Codigo_Tipo'][i] in ('DESCONHECIDO','EMENDA'):
        naovaiter+=1
    else:
  
        naotem+=1
        #oid=i
print(tem)
print(naotem)
print(naovaiter)
#print(oid)  